## 等時マップ

In [8]:
import requests
import ipywidgets as widgets
from ipywidgets import interact, Select, Layout
import json
from mapboxgl_notebook.map import MapboxMap
from mapboxgl_notebook.sources import GeoJSONSource
from mapboxgl_notebook.layers import PointCircleLayer, LineStringLineLayer, PolygonFillLayer
from mapboxgl_notebook.properties import Paint
from shapely.geometry import LineString
import geojson
import warnings
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

warnings.simplefilter('ignore')

In [19]:
def inchrone(address="八千代台駅",ways="ぶーぶ",minute=10):
    def get_lat_lon_from_address(address_l):
        url = 'http://www.geocoding.jp/api/'
        latlons = []
        for address in tqdm(address_l):
            payload = {'q': address}
            r = requests.get(url, params=payload)
            ret = BeautifulSoup(r.content,'lxml')
            if ret.find('error'):
                raise ValueError(f"Invalid address submitted. {address}")
            else:
                lat = ret.find('lat').string
                lon = ret.find('lng').string
                latlons.append([lat,lon])
        return latlons

    latlon = get_lat_lon_from_address([address])
    latitude = latlon[0][0]
    longtitude = latlon[0][1]

    if ways == "ぶーぶ":
        ways = "driving"
    elif ways == "チャリ":
        ways = "cycling"
    elif ways == "てくてく":
        ways = "walking"
    minute = str(minute)
    geometry = requests.get("https://api.mapbox.com/isochrone/v1/mapbox/"+ways+"/"+ longtitude + "," + latitude +"?contours_minutes=" + minute + "&polygons=true&access_token=pk.eyJ1IjoiZnJrdzEyMjIiLCJhIjoiY2s4aXlqaW9uMGI2dTNscG1mMDA5eW0wZyJ9.7wGREzjjAHpiRBJyI3jDyg").json()
    source = GeoJSONSource(geometry, source_id='name')
    paint = Paint(
        fill_color='rgba(254, 202, 68, 0.2)',
        fill_outline_color='rgba(0, 0, 0, 1)'
    )
    layer = PolygonFillLayer(source, paint=paint)
    mapbox_map = MapboxMap(
        center=[longtitude,latitude],
        zoom=13,
        access_token="pk.eyJ1IjoiZnJrdzEyMjIiLCJhIjoiY2s4aXlqaW9uMGI2dTNscG1mMDA5eW0wZyJ9.7wGREzjjAHpiRBJyI3jDyg",
        sources=[source],
        layers=[layer],
        style="mapbox://styles/frkw1222/cke9pbiak0zgr1apadswmw9az"
    )
    mapbox_map.show()

In [20]:
style = {'description_width': '200px'}
address = widgets.Text(
    value='八千代台駅',
    style=style,
    placeholder='なんか入れろや',
    description='どこから行くの？　:',
    disabled=False,
    layout=Layout(width='80%')
)
ways = Select(description='何で行くの？　:', options=["ぶーぶ","チャリ","てくてく"], rows=3,style=style,)
minute = widgets.Text(
    value='10',
    style=style,
    placeholder='なんか入れろたほうが身のためやで',
    description='何分で行くの？ :',
    disabled=False,
    layout=Layout(width='80%')
)
interact(inchrone, address=address, ways=ways, minute=minute)

interactive(children=(Text(value='八千代台駅', description='どこから行くの？\u3000:', layout=Layout(width='80%'), placehold…

<function __main__.inchrone(address='八千代台駅', ways='ぶーぶ', minute=10)>